In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import torch
import matplotlib.pyplot as plt

# Callbacks

### Callbacks as GUI Events

In [3]:
import ipywidgets as widgets

From the [ipywidget docs](https://ipywidgets.readthedocs.io/en/stable/examples/Widget%20Events.html):

- *the button widget is used to handle mouse clicks. The on_click method of the Button can be used to register function to be called when the button is clicked*

In [8]:
from IPython.display import display
button = widgets.Button(description="Click Me!")
output = widgets.Output()

display(button, output)

def on_button_clicked(b):
    with output:
        print("Button clicked.")

button.on_click(on_button_clicked)

Button(description='Click Me!', style=ButtonStyle())

Output()

### Creating your own Callback

In [25]:
from time import sleep

In [26]:
def slow_calculation():
    res = 0
    for i in range(5):
        res += i*i
        sleep(1)
    return res

In [28]:
slow_calculation()

30

In [29]:
def slow_calculation(cb=None):
    res = 0
    for i in range(5):
        res += i*i
        sleep(1)
        if cb: cb(i)
    return res

In [30]:
def show_progress(epoch):
    print(f'Awesome!! We\'ve finished epoch {epoch}')

In [31]:
slow_calculation(show_progress)

Awesome!! We've finished epoch 0
Awesome!! We've finished epoch 1
Awesome!! We've finished epoch 2
Awesome!! We've finished epoch 3
Awesome!! We've finished epoch 4


30

### Lambdas and partials

In [33]:
slow_calculation(lambda o: print(f'Awesome!! We\'ve finished epoch {o}'))

Awesome!! We've finished epoch 0
Awesome!! We've finished epoch 1
Awesome!! We've finished epoch 2
Awesome!! We've finished epoch 3
Awesome!! We've finished epoch 4


30

In [34]:
def show_progress(exclamation, epoch):
    print(f'{exclamation} We\'ve finished epoch {epoch}')

In [36]:
slow_calculation(lambda o: show_progress('Super!', o))

Super! We've finished epoch 0
Super! We've finished epoch 1
Super! We've finished epoch 2
Super! We've finished epoch 3
Super! We've finished epoch 4


30

In [37]:
def make_show_progress(exclamation):
    _inner = lambda o: print(f'{exclamation}, We\'ve finished epoch {o}')
    return _inner

In [38]:
slow_calculation(make_show_progress('Nice!'))

Nice!, We've finished epoch 0
Nice!, We've finished epoch 1
Nice!, We've finished epoch 2
Nice!, We've finished epoch 3
Nice!, We've finished epoch 4


30

In [39]:
f2  = make_show_progress('Geil!')

In [40]:
slow_calculation(f2)

Geil!, We've finished epoch 0
Geil!, We've finished epoch 1
Geil!, We've finished epoch 2
Geil!, We've finished epoch 3
Geil!, We've finished epoch 4


30

In [41]:
from functools import partial

In [42]:
slow_calculation(partial(show_progress, 'Fantastic!'))

Fantastic! We've finished epoch 0
Fantastic! We've finished epoch 1
Fantastic! We've finished epoch 2
Fantastic! We've finished epoch 3
Fantastic! We've finished epoch 4


30

### Callbacks as callable classes

In [44]:
class ProgressShowCallback():
    def __init__(self, exclamation="Awesome!"): self.exclamation = exclamation
    def __call__(self, epoch): print(f'{self.exclamation}, We\'ve finished epoch {epoch}')

In [45]:
cb = ProgressShowCallback()

In [46]:
slow_calculation(cb)

Awesome!, We've finished epoch 0
Awesome!, We've finished epoch 1
Awesome!, We've finished epoch 2
Awesome!, We've finished epoch 3
Awesome!, We've finished epoch 4


30

### Multiple callback functions, `*args` and `**kwargs`

In [47]:
def f(*args, **kwargs): print(f'args: {args}, kwargs: {kwargs}')

In [48]:
f(3, 'a', thing1='hello')

args: (3, 'a'), kwargs: {'thing1': 'hello'}


In [49]:
def slow_calculation(cb=None):
    res=0
    for i in range(5):
        if cb: cb.before_call(i)
        res += i*i
        sleep(1)
        if cb: cb.after_call(i, val=res)
    return res

In [50]:
class PrintStepCallback():
    def __init__(self): pass
    def before_call(self, *args, **kwargs): print(f'About to start')
    def after_call(slef, *args, **kwargs): print(f'started')

In [51]:
slow_calculation(PrintStepCallback)

About to start
started
About to start
started
About to start
started
About to start
started
About to start
started


30

### MOdifying behaviour